In [1]:
import numpy as np 
import pandas as pd
from spacy.lang.en.stop_words import STOP_WORDS
from gensim import corpora
from operator import itemgetter
from gensim.similarities import MatrixSimilarity
import string
import spacy
import re
import gensim 

C:\Anaconda\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
data = pd.read_csv('serchresult.csv')
data.head()

,Unnamed: 0,v_id,v_title,Keyword,publishedDate,v_description,viewCount,likeCount,dislikeCount,favoriteCount,commentCount
0,0,-ETQ97mXXF0,Data Science Full Course - Learn Data Science ...,Python for data science,2019-08-18T08:30:02Z,"🔥 Data Science Master Program (Use Code ""𝐘𝐎𝐔𝐓𝐔...",1358779,31487.0,393.0,0,376.0
1,1,X3paOmcrTjQ,Data Science In 5 Minutes | Data Science For B...,Python for data science,2018-12-04T14:30:01Z,🔥Free Data Science with Python course: https:/...,1832910,26212.0,974.0,0,947.0
2,2,u2zsY-2uZiE,Data Science Tutorial | Data Science for Begin...,Python for data science,2020-02-03T12:56:37Z,🔥Get the free certificate of completion for th...,646074,21656.0,262.0,0,469.0
3,3,xC-c7E5PK0Y,What REALLY is Data Science? Told by a Data Sc...,Data science,2018-06-23T01:51:50Z,Interested in Data Science? Start with learnin...,2071496,80087.0,1048.0,0,2705.0
4,4,XnNzck5-HdQ,Complete Roadmap to Become a Data Scientist,Python for data science,2020-09-23T14:35:16Z,How to become a Data Scientist: Become a self ...,352663,22723.0,250.0,0,2501.0


In [3]:
data1=data.drop(['Unnamed: 0','v_id','Keyword','publishedDate','viewCount','likeCount','dislikeCount','commentCount'],axis=1)

In [4]:
data1.head()

,v_title,v_description,favoriteCount
0,Data Science Full Course - Learn Data Science ...,"🔥 Data Science Master Program (Use Code ""𝐘𝐎𝐔𝐓𝐔...",0
1,Data Science In 5 Minutes | Data Science For B...,🔥Free Data Science with Python course: https:/...,0
2,Data Science Tutorial | Data Science for Begin...,🔥Get the free certificate of completion for th...,0
3,What REALLY is Data Science? Told by a Data Sc...,Interested in Data Science? Start with learnin...,0
4,Complete Roadmap to Become a Data Scientist,How to become a Data Scientist: Become a self ...,0


In [5]:
data1["v_title1"]=data.v_title
data1["v_description1"]=data.v_description

In [6]:
spacy_nlp = spacy.load('en_core_web_sm')

#create list of punctuations and stopwords
punctuations = string.punctuation
stop_words = spacy.lang.en.stop_words.STOP_WORDS

def spacy_tokenizer(sentence):
 
    #remove distracting single quotes
    sentence = re.sub('\'','',sentence)

    #remove digits adnd words containing digits
    sentence = re.sub('\w*\d\w*','',sentence)

    #replace extra spaces with single space
    sentence = re.sub(' +',' ',sentence)

    #remove unwanted lines starting from special charcters
    sentence = re.sub(r'\n: \'\'.*','',sentence)
    sentence = re.sub(r'\n!.*','',sentence)
    sentence = re.sub(r'^:\'\'.*','',sentence)
    
    #remove non-breaking new line characters
    sentence = re.sub(r'\n',' ',sentence)
    
    #remove punctunations
    sentence = re.sub(r'[^\w\s]',' ',sentence)
    
    #creating token object
    tokens = spacy_nlp(sentence)
    
    #lower, strip and lemmatize
    tokens = [word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in tokens]
    
    #remove stopwords, and exclude words less than 2 characters
    tokens = [word for word in tokens if word not in stop_words and word not in punctuations and len(word) > 2]
    
    #return tokens
    return tokens

In [7]:
data1['v_title1'] = data1['v_title1'].map(lambda x: spacy_tokenizer(x))

In [8]:
data1["v_title1"][89]

['skills', 'data', 'analyst', 'data', 'analyst', 'edureka']

In [9]:
v_title1=data1["v_title1"]
v_title1

0       [data, science, course, learn, data, science, ...
1       [data, science, minutes, data, science, beginn...
2       [data, science, tutorial, data, science, begin...
3                  [data, science, tell, data, scientist]
4                    [complete, roadmap, data, scientist]
                              ...                        
2353    [hypothesis, testing, types, selection, correc...
2354                           [test, hypothesis, telugu]
2355    [level, cfa, quant, hypothesis, testing, lecture]
2356    [hypothesis, types, hypothesis, null, amp, alt...
2357    [statistical, hypothesis, testing, data, scien...
Name: v_title1, Length: 2358, dtype: object

In [10]:
v_title1[2356]

['hypothesis',
 'types',
 'hypothesis',
 'null',
 'amp',
 'alternative',
 'research',
 'methodology']

In [11]:
word = corpora.Dictionary(v_title1)

In [12]:
print(word)

Dictionary(2338 unique tokens: ['beginner', 'course', 'data', 'edureka', 'hour']...)


In [13]:
corpus = [word.doc2bow(desc) for desc in v_title1 ]

In [14]:
word_frequencies = [[(word[id], frequency) for id, frequency in line] for line in corpus[0:2357]]
print(word_frequencies)

[[('beginner', 1), ('course', 1), ('data', 3), ('edureka', 1), ('hour', 1), ('learn', 1), ('science', 3)], [('beginner', 1), ('data', 3), ('science', 3), ('minutes', 1), ('simplilearn', 1)], [('beginner', 1), ('course', 1), ('data', 3), ('hour', 1), ('science', 3), ('python', 1), ('tutorial', 1)], [('data', 2), ('science', 1), ('scientist', 1), ('tell', 1)], [('data', 1), ('scientist', 1), ('complete', 1), ('roadmap', 1)], [('data', 1), ('science', 1), ('hindi', 1)], [('data', 2), ('science', 1), ('scientist', 1), ('advice', 1), ('amp', 1), ('artificial', 1), ('career', 1), ('intelligence', 1), ('life', 1), ('real', 1)], [('data', 1), ('learn', 1), ('science', 1), ('smartly', 1)], [('beginner', 1), ('course', 1), ('data', 1), ('learn', 1), ('science', 1), ('tutorial', 1)], [('beginner', 1), ('data', 3), ('edureka', 1), ('science', 3), ('minutes', 1)], [('course', 1), ('data', 1), ('scientist', 1), ('complete', 1), ('hindi', 1), ('degree', 1), ('detail', 1), ('future', 1), ('highest', 1

In [15]:
tfidf_model = gensim.models.TfidfModel(corpus, id2word=word)
lsi_model = gensim.models.LsiModel(tfidf_model[corpus], id2word=word, num_topics=300)

In [16]:
gensim.corpora.MmCorpus.serialize('tfidf_model_mm', tfidf_model[corpus])
gensim.corpora.MmCorpus.serialize('lsi_model_mm',lsi_model[tfidf_model[corpus]])

In [17]:
#Load the indexed corpus
tfidf_corpus = gensim.corpora.MmCorpus('tfidf_model_mm')
lsi_corpus = gensim.corpora.MmCorpus('lsi_model_mm')

In [18]:
video_index = MatrixSimilarity(lsi_corpus, num_features = lsi_corpus.num_terms)

In [21]:
class video_rec:
    search_term="Data Science"
    
    def __name__(self,search_term):
        self.search_term=search_term
        query_bow = word.doc2bow(spacy_tokenizer(search_term))
        query_tfidf = tfidf_model[query_bow]
        query_lsi = lsi_model[query_tfidf]

        video_index.num_best = 10

        video_list = video_index[query_lsi]

        video_list.sort(key=itemgetter(1), reverse=True)
        video_names = []

        for j, video in enumerate(video_list):

            video_names.append (
                {   
                    'video_link':"https://www.youtube.com/watch?v=" + data['v_id'][video[0]],
                    'video_Title': data['v_title'][video[0]],
                    'video_des':  data['v_description'][video[0]],
                    'video_view_count':data['viewCount'][video[0]],
                    'video_like_count':data['likeCount'][video[0]],
                    'video_dislike_count':data['dislikeCount'][video[0]],
                    'video_comment_count':data['commentCount'][video[0]]
                }

            )
            if j == (video_index.num_best-1):
                break

        return pd.DataFrame(video_names, columns=['video_link','video_Title','video_des','video_view_count','video_like_count','video_dislike_count','video_comment_count'])


In [28]:
# search for movie tiles that are related to below search parameters
video_rec()

In [51]:
video_rec('python for data science to beginers')

,video_link,video_Title,video_des,video_view_count,video_like_count,video_dislike_count,video_comment_count
0,https://www.youtube.com/watch?v=xMpllQLKDZ4,Python for Data Science | Data Science with Py...,( Python Data Science Training: https://www.ed...,74088,948.0,27.0,36.0
1,https://www.youtube.com/watch?v=mkv5mxYu0Wk,Data Science With Python | Python for Data Sci...,This Data Science with Python Tutorial will he...,177602,2361.0,45.0,568.0
2,https://www.youtube.com/watch?v=mNdbcHECGN4,Python for Data Science | Data Science With Py...,🔥Intellipaat Python for Data Science Course: h...,418697,7378.0,257.0,388.0
3,https://www.youtube.com/watch?v=Q4ap8MXd4T4,Data Science with Python | Python for Data Sci...,🔥Intellipaat Data Science with Python course: ...,66175,1310.0,60.0,81.0
4,https://www.youtube.com/watch?v=k2P_pHQDlp0,AI VS ML VS DL VS Data Science,Please do subscribe my vlogging channel for mo...,1026659,35584.0,689.0,1136.0
5,https://www.youtube.com/watch?v=IZubolzAGKQ,Python Packages for Data Science|Top 5 Python ...,Python Packages for Data Science|Top 5 Python ...,1087,47.0,1.0,20.0
6,https://www.youtube.com/watch?v=edvg4eHi_Mw,Python for Data Science | Data Science with Py...,Looking for a career upgrade & better salary? ...,385848,12150.0,142.0,629.0
7,https://www.youtube.com/watch?v=esPfw_wya6E,Data Science with Python | Python For Data Sci...,🔵 Intellipaat Data Science with Python course...,26250,868.0,53.0,29.0
8,https://www.youtube.com/watch?v=m2pUda4UcMI,Data Analytics with Python | Python for Data A...,🔵 Intellipaat Data Analytics training course: ...,12259,446.0,11.0,14.0
9,https://www.youtube.com/watch?v=Vn7-jOUbVpw,Data Science Full Course | Data Science With P...,This video on Data Science with Python full co...,13732,599.0,4.0,18.0


In [52]:
video_rec('introduction to my sql')

,video_link,video_Title,video_des,video_view_count,video_like_count,video_dislike_count,video_comment_count
0,https://www.youtube.com/watch?v=k5WZ9MJTA_Y,sql introduction tutorial,sql introduction in database systems,320780,4171.0,242.0,146.0
1,https://www.youtube.com/watch?v=DWuMCKLX1jo,SQL Introduction || 1||SQL in telugu,What is sql and history,21715,444.0,5.0,56.0
2,https://www.youtube.com/watch?v=y2CICRujsVw,Oracle - SQL - Introduction,Oracle - SQL - Introduction\nWatch more Videos...,272854,2330.0,67.0,56.0
3,https://www.youtube.com/watch?v=xM51sGAo3Jg,Sql vs tsql vs plsql,sql vs pl/sql vs mysql | sql vs tsql | pl sql ...,23055,686.0,25.0,51.0
4,https://www.youtube.com/watch?v=YufocuHbYZo,SQL SELECT Tutorial |¦| SQL Tutorial |¦| SQL...,The SELECT statement in SQL is how you fetch d...,122616,4414.0,42.0,268.0
5,https://www.youtube.com/watch?v=wkOD6mbXc2M,SQL Course | SQL Tutorial For Beginners | Lear...,👉This SQL course video is an SQL tutorial for ...,624804,11330.0,288.0,537.0
6,https://www.youtube.com/watch?v=PApdxLedQoc,SQL For Beginners | SQL Tutorial For Beginners...,Looking for a career upgrade & a better salary...,30645,690.0,19.0,42.0
7,https://www.youtube.com/watch?v=LGTbdjoEBVM,SQL Course | SQL Training | SQL Tutorial For B...,"🔵 In this live session on the SQL course, fir...",110269,2517.0,60.0,138.0
8,https://www.youtube.com/watch?v=BPHAr4QGGVE,SQL Full Course | SQL Tutorial For Beginners |...,** MYSQL DBA Certification Training (Use code:...,887292,13606.0,238.0,113.0
9,https://www.youtube.com/watch?v=zbMHLJ0dY4w,SQL Basics for Beginners | Learn SQL | SQL Tut...,"🔥 Edureka Online Training (Use Code ""𝐘𝐎𝐔𝐓𝐔𝐁𝐄𝟐𝟎...",793184,15317.0,366.0,154.0


In [94]:
video_rec.abc()

TypeError: abc() missing 1 required positional argument: 'search_term'

In [54]:
import pickle 

In [81]:
pickle.dump(video_rec, open('video.pkl', 'wb'))

In [90]:
pickle_in = open('video.pkl', 'rb') 
video_rec = pickle.load(pickle_in)

In [92]:
video_rec.abc('my sql')

,video_link,video_Title,video_des,video_view_count,video_like_count,video_dislike_count,video_comment_count
0,https://www.youtube.com/watch?v=xM51sGAo3Jg,Sql vs tsql vs plsql,sql vs pl/sql vs mysql | sql vs tsql | pl sql ...,23055,686.0,25.0,51.0
1,https://www.youtube.com/watch?v=YufocuHbYZo,SQL SELECT Tutorial |¦| SQL Tutorial |¦| SQL...,The SELECT statement in SQL is how you fetch d...,122616,4414.0,42.0,268.0
2,https://www.youtube.com/watch?v=wkOD6mbXc2M,SQL Course | SQL Tutorial For Beginners | Lear...,👉This SQL course video is an SQL tutorial for ...,624804,11330.0,288.0,537.0
3,https://www.youtube.com/watch?v=PApdxLedQoc,SQL For Beginners | SQL Tutorial For Beginners...,Looking for a career upgrade & a better salary...,30645,690.0,19.0,42.0
4,https://www.youtube.com/watch?v=LGTbdjoEBVM,SQL Course | SQL Training | SQL Tutorial For B...,"🔵 In this live session on the SQL course, fir...",110269,2517.0,60.0,138.0
5,https://www.youtube.com/watch?v=BPHAr4QGGVE,SQL Full Course | SQL Tutorial For Beginners |...,** MYSQL DBA Certification Training (Use code:...,887292,13606.0,238.0,113.0
6,https://www.youtube.com/watch?v=DWuMCKLX1jo,SQL Introduction || 1||SQL in telugu,What is sql and history,21715,444.0,5.0,56.0
7,https://www.youtube.com/watch?v=zbMHLJ0dY4w,SQL Basics for Beginners | Learn SQL | SQL Tut...,"🔥 Edureka Online Training (Use Code ""𝐘𝐎𝐔𝐓𝐔𝐁𝐄𝟐𝟎...",793184,15317.0,366.0,154.0
8,https://www.youtube.com/watch?v=rB382wSHuTg,SQL Tutorial for Beginners | SQL Full Course |...,🔥 Get the free certificate of completion for t...,26212,674.0,13.0,46.0
9,https://www.youtube.com/watch?v=2XB5CddzEaM,SQL commands and PL/SQL programs complete in t...,#edusoft4u #edusoftlearningsystems\n#softwaret...,26923,501.0,16.0,37.0


In [59]:
pickle_in.close()